In [1]:
import json
import requests
import pandas as pd
from config import CLIENT_ID
from config import CLIENT_SECRET
from config import password
from sqlalchemy import create_engine

In [2]:
AUTH_URL = 'https://accounts.spotify.com/api/token'

# POST
auth_response = requests.post(AUTH_URL, {
    'grant_type': 'client_credentials',
    'client_id': CLIENT_ID,
    'client_secret': CLIENT_SECRET,
})

# convert the response to JSON
auth_response_data = auth_response.json()

# save the access token
access_token = auth_response_data['access_token']

#set header
headers = {
    'Authorization': 'Bearer {token}'.format(token=access_token)
}

In [39]:
playlist_json = requests.get('https://api.spotify.com/v1/playlists/5zrICw4pXEoaUQ80xUIPz4', headers=headers).json()

tracks = []
artists = []
album = []
duration = []
track_id =[]
artist_id = []
for x in range(0, len(playlist_json['tracks']['items'])):
    tracks.append(playlist_json['tracks']['items'][x]['track']['name'])
    artists.append(playlist_json['tracks']['items'][x]['track']['album']['artists'][0]['name'])
    album.append(playlist_json['tracks']['items'][x]['track']['album']['name'])
    duration.append(playlist_json['tracks']['items'][x]['track']['duration_ms'])
    track_id.append(playlist_json['tracks']['items'][x]['track']['id'])
    artist_id.append(playlist_json['tracks']['items'][x]['track']['artists'][0]['id'])

In [41]:
dict = {
    'song' : tracks,
    'artist' : artists,
    'album': album,
    'length' : duration,
    'track_id': track_id,
    'artist_id': artist_id
}

Playlist_df = pd.DataFrame(dict)

In [44]:
Playlist_df = Playlist_df[['track_id', 'song', 'artist','artist_id', 'album', 'length']]

In [45]:
Playlist_df

,track_id,song,artist,artist_id,album,length
0,3nXUfNbkv8ikaSdHwEp0oY,DDD,EXID,1xs6WFotNQSXweo0GXrS0O,Full Moon,205821
1,3aW0ds4A4tSQDIp75FqWTo,FAKE LOVE,BTS,3Nrfpe0tUJi4K4DXYWgMUX,Love Yourself 轉 'Tear',242333
2,3o8QzWsiiqTUVgBZfHgF58,Cherry Bomb,NCT 127,7f4ignuCJhLXfZ9giKT7rH,NCT #127 CHERRY BOMB– The 3rd Mini Album,236667
3,6gGp4A2a6TUWhzYrhQQbk4,BLING BLING,iKON,5qRSs6mvI17zrkJpOHkCoM,NEW KIDS : BEGIN,219225
4,5EzitieoPnjyKHAq0gfRMa,Ko Ko Bop,EXO,3cjEqqelV9zb4BYE3qDQ4O,THE WAR - The 4th Album,190422
...,...,...,...,...,...,...
79,2V4Fx72svQRxrFvNT1eq5f,THE BADDEST,K/DA,4gOc8TsQed9eqnqJct2c5v,THE BADDEST,162600
80,4qMt92hgQlJMyJjdI0Nw2h,GUNSHOT,KARD,2JhAlkmukNvarUpGhTFXUQ,KARD 1st Single ‘Way With Words’,181480
81,6juLaduD4STCUDWT0AYun4,MORE,K/DA,4gOc8TsQed9eqnqJct2c5v,MORE,217088
82,7wDVvxMUdW5MtJUqFtuXUz,Bicycle,CHUNG HA,2PSJ6YriU7JsFucxACpU7Y,Querencia,179156


In [16]:
rds_connection_string = f'postgres:{password}@localhost:5432/project_db'
engine = create_engine(f'postgresql://{rds_connection_string}')

In [17]:
engine.table_names()

['playlist_lawrence']

In [46]:
Playlist_df.to_sql(name='playlist_lawrence', con=engine, if_exists='append', index=True)

In [36]:
playlist_json['tracks']['items'][0]['track']['artists'][0]['id']

'1xs6WFotNQSXweo0GXrS0O'